<a href="https://colab.research.google.com/github/MengOonLee/Deep_learning/blob/master/TFDS/SplitsAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring the Splits API

## Setup

We'll start by importing TensorFlow and TensorFlow Datasets.

In [ ]:
%%bash
pip install --no-cache-dir -qU pip wheel
pip install --no-cache-dir -qU numpy pandas matplotlib seaborn scikit-learn
pip install --no-cache-dir -qU tensorflow-cpu tensorflow-datasets
pip check

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import warnings
warnings.filterwarnings('ignore')

import numpy as np
np.random.seed(42)

import pandas as pd
import json

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style('whitegrid')
sns.set(font='DejaVu Sans')

import tensorflow as tf
tf.keras.utils.set_random_seed(42)
tf.get_logger().setLevel('ERROR')

import tensorflow_datasets as tfds

print("\u2022 Using TensorFlow Version:", tf.__version__)

• Using TensorFlow Version: 2.11.0


## Exploring the Splits API

In [2]:
# Distinct splits
# The full `train` split and the full `test` split as two distinct datasets. 
train_ds, test_ds = tfds.load(name='mnist:3.*.*',
    split=[tfds.Split.TRAIN, tfds.Split.TEST])

print(len(list(train_ds)))
print(len(list(test_ds)))

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
60000
10000


In [4]:
# Merging
# The full `train` and `test` splits, concatenated together.
combined_ds = tfds.load(name='mnist:3.*.*',
    split='train+test')

print(len(list(combined_ds)))

70000


In [5]:
# Slicing by index
ds = tfds.load(name='mnist:3.*.*',
    split=tfds.Split.TRAIN[:10000])

print(len(list(ds)))

60000


With the slicing API we can use strings to specify the slicing instructions. For example, in the cell below we will merge the training and test sets by passing the string `’train+test'` to the `split` argument.

In [ ]:
combined = tfds.load('mnist:3.*.*', split='train+test')

print(len(list(combined)))

We can also use Python style list slicers to specify the data we want. For example, we can specify that we want to take the first 10,000 records of the `train` split with the string `'train[:10000]'`, as shown below:

In [ ]:
first10k = tfds.load('mnist:3.*.*', split='train[:10000]')

print(len(list(first10k)))

It also allows us to specify the percentage of the data we want to use. For example, we can select the first 20\% of the training set with the string `'train[:20%]'`, as shown below:

In [ ]:
first20p = tfds.load('mnist:3.*.*', split='train[:20%]')

print(len(list(first20p)))

We can see that `first20p` contains 12,000 records, which is indeed 20\% the total number of records in the training set. Recall that the training set contains 60,000 records. 

Because the slices are string-based we can use loops, like the ones shown below, to slice up the dataset and make some pretty complex splits. For example, the loops below create 10 complimentary validation and training sets (each loop returns a list with 5 data sets).

In [ ]:
val_ds = tfds.load('mnist:3.*.*', split=['train[{}%:{}%]'.format(k, k+20) for k in range(0, 100, 20)])

train_ds = tfds.load('mnist:3.*.*', split=['train[:{}%]+train[{}%:]'.format(k, k+20) for k in range(0, 100, 20)])

In [ ]:
val_ds

In [ ]:
train_ds

In [ ]:
print(len(list(val_ds)))
print(len(list(train_ds)))

We can also compose new datasets by using pieces from different splits. For example, we can create a new dataset from the first 10\% of the test set and the last 80\% of the training set, as shown below.

In [ ]:
composed_ds = tfds.load('mnist:3.*.*', split='test[:10%]+train[-80%:]')

print(len(list(composed_ds)))